# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию
<a id='step1'></a>

In [4]:
import pandas as pd

credit_dataset = pd.read_csv('...')
credit_dataset.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
# КОД РЕВЬЮЕРА

credit_dataset = pd.read_csv('...')

Исследуем уникальные значения в каждом из столбцов:

In [72]:
for column in credit_dataset.columns:
    column_values = credit_dataset[column].unique()
    print(f'{column} : {column_values}')
    print('-' * 50)

children : [ 1  0  3  2 -1  4 20  5]
--------------------------------------------------
days_employed : [-8437.67302776 -4024.80375385 -5623.42261023 ... -2113.3468877
 -3112.4817052  -1984.50758853]
--------------------------------------------------
dob_years : [42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75]
--------------------------------------------------
education : ['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']
--------------------------------------------------
education_id : [0 1 2 3 4]
--------------------------------------------------
family_status : ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']
-------------------------------------------

Сразу бросаются в глаза следующие странности:
- отрицательные значения в столбце `days_employed`;
- значение `XNA` для категории `gender`;
- аномальный возраст в категории `dob_years` - значение `0`;
- аномальное количество детей в категории `children` - значения `-1` и `20`

Теперь попытаемся узнать больше о столбце `days_employed`, который, как мы увидели выше, содержит в себе отрицательные значения:

In [73]:
credit_dataset['days_employed'].describe()

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

Сложно сказать, что представляют собой значения в этом столбце `days_employed`: максимальное значение в нём равно `401755`, что соответствует совершенно нереальным ~1100 годам стажа; минимальное же значение оказалось по какой-то причине отрицательным (`-18388`).
Кроме того, настораживает использование вещественного типа данных для представления значений этой категории ("стаж в днях"): даже если представить, что это большое количество знаков после запятой в действительности отражает какие-то малые доли рабочего дня, то всё равно выглядит маловероятным, что это значение могло быть получено/измеряно с такой точностью и сразу для нескольких тысяч людей.

Наконец, получим сводную информацию по таблице:

In [74]:
credit_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**
Глядя на первые 10 строк датасета, большинство значений столбцов выглядят логичными и понятными. Сразу видно, что значения столбца `education` и `family_status` будут нуждаться в приведении к нижнему регистру, а значения столбца `purpose` - в какой-то более сложной лингвистической обработке, чтобы свести, например, значения "_сыграть свадьбу_" и "_на проведение свадьбы_" к некоторому общему. Кроме того, были замечены некоторые странности в значениях некоторых столбцов (аномальные значения), о которых было написано выше.

Также кажется необходимым отметить, что несмотря на разницу в регистре в столбце `education`, значения `education_id` проставлены без учёта регистра. Так, значения `среднее`, `Среднее` и `СРЕДНЕЕ` имеют одно и то же значение `education_id`, равное 1. Этот факт позволит нам далее выполнить приведение к нижнему регистру значений столбца `education` без опасений о том, что это нарушит согласованность между столбцами `education` и `education_id`.

Краткая сводка о `DataFrame`, полученная вызовом метода `info()`, показала, что столбцы `days_employed` и  `total_income` содержат пропущенные значения (`null`). Всего же в датасете `21525` строк.

## Шаг 2. Предобработка данных

Сделаем замену значения `XNA` в столбце `gender` на более частотный пол, для чего предварительно сделаем группировку:

In [75]:
credit_dataset.groupby('gender')['gender'].count()

gender
F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Как можно видеть, женщин в наборе данных аж вдвое больше, поэтому выставим в качестве пола вместо `XNA` значение `F`:

In [76]:
credit_dataset.loc[credit_dataset['gender'] == 'XNA', 'gender'] = 'F'

Теперь осуществим замену аномальных значений в столбце `children`.
На мой взгляд, появление значения `-1` могло бы означать, что у клиента отсутствуют дети, но он посчитал (почему-то) более логичным использовать отрицательное значение как обозначение отсутствия, а не ноль. Выполним замену значения на `0` в этом случае:

In [77]:
credit_dataset.loc[credit_dataset['children'] == -1, 'children'] = 0

Аномальное значение в `20` детей возникло скорее всего в результате нежелания клиента указать реальное количество детей, поэтому мне кажется разумным выполнить замену этого значения `20` на медианное количесто детей во всем датасете (медиану я бы взял потому, чтобы избежать влияния выбросов со значением `20`):

In [78]:
credit_dataset.loc[credit_dataset['children'] == 20, 'children'] = int(credit_dataset['children'].median())

Замену аномального возраста в `0` лет в категории `dob_years` будем выполнять, используя среднее значение, но не по всему датасету, а по мужчинам и женщинам отдельно, предварительно подсчитав медианные возраста для этих двух категорий людей:

In [79]:
#credit_dataset.groupby('gender')['dob_years'].median()

In [80]:
#credit_dataset.loc[(credit_dataset['dob_years'] == 0) & (credit_dataset['gender'] == 'F'), 'dob_years'] = 44
#credit_dataset.loc[(credit_dataset['dob_years'] == 0) & (credit_dataset['gender'] == 'M'), 'dob_years'] = 40

### Обработка пропусков

In [81]:
credit_dataset.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Примечательно, что количество пропущенных значений в столбцах `days_employed` и  `total_income` в точности совпадают и равны `2174`. Проверим, в каких строках имеются пропуски для `days_employed`, и в каких строках - для `total_income`. А точнее, выведем все такие строки, где пропущенными являются значения __в обоих этих столбцах одновременно__:

In [82]:
credit_dataset[credit_dataset['days_employed'].isnull() & credit_dataset['total_income'].isnull()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


И здесь ровно `2174`. Таким образом, в нашем датасете пропуски и в том, и в другом столбце всегда наблюдаются одновременно.

Всего же в нашем датасете около 10% строк имеют пропуски (2174 / 21525 * 100 = 10,09%).

Для заполнения пропусков в столбце `total_income`, применим более хитрую стратегию: сначала осуществим деление всех клиентов на интервалы по возрасту (`dob_years`), используя 10 лет в качестве длины интервала. Затем, проведём группировку, чтобы в рамках каждого возрастного интервала выделить подгруппы, исходя из пола (`gender`) и типа занятости клиента (`income_type`). Таким образом, получатся группы следующего вида:
- _мужчины, сотрудники, возрастом 21-30_
- _женщины, сотрудники, возрастом 21-30_
- _мужчины, сотрудники, возрастом 31-40_
- ...

Для выполнения деления на интервалы по возрасту, воспользуемся вспомогательной функцией `age_range()`, которую применим к столбцу `dob_years` с помощью метода `Series.apply()`; результат при этом запишем в новый столбец `age_interval` нашей таблицы:

In [83]:
def age_range(age):
    """Возвращает возрастной интервал (в виде строки) в зависимости от возраста, переданного в аргумент 'age'
    """
    if age <= 20:
     return '0-20'
    elif 20 < age <= 30:
     return '21-30'
    elif 30 < age <= 40:
     return '31-40'
    elif 40 < age <= 50:
     return '41-50'
    elif 50 < age <= 60:
     return '51-60'
    elif 60 < age <= 70:
     return '61-70'
    elif 70 < age <= 80:
     return '71-80'
    elif 80 < age <= 90:
     return '81-90'
    else:
     return '90+'

Применяем нашу функцию к таблице, создавая тем самым столбец с возрастными интервалами (`age_interval`):

In [84]:
credit_dataset['age_interval'] = credit_dataset['dob_years'].apply(age_range)

Имея столбцы `income_type`, `gender` и `age_interval`, выполним группировку по ним, чтобы использовать средние доходы внутри каждой из получившихся групп для заполнения пропусков в столбце с доходом (`total_income`).

Для тренировки, создадим как саму группировку через `DataFrame.groupby()` (её будем использовать для извлечения средних доходов по группам), так и сводную таблицу через `pandas.pivot_table()` (её будем использовать для наглядной визуализации получившихся в результате групп):

In [85]:
def create_groupby_and_pivot(df):
    """Возвращает объекты группировки и сводной таблицы
    """
    # Создадим группировку ...
    groupby = credit_dataset.groupby(['income_type', 'gender', 'age_interval'])['total_income'].mean()

    # ... и аналогичную по смысловому наполнению сводную таблицу
    pivot = pd.pivot_table(credit_dataset,
                           values='total_income',
                           index=['income_type', 'gender'],
                           columns='age_interval')
    return groupby, pivot

incomes_groupby, incomes_pivot = create_groupby_and_pivot(credit_dataset)
incomes_pivot

age_interval                     0-20          21-30          31-40  \
income_type     gender                                                
безработный     F                 NaN            NaN            NaN   
                M                 NaN            NaN   59956.991984   
в декрете       F                 NaN            NaN   53829.130729   
госслужащий     F       119517.250360  138796.667735  158070.269384   
                M       186784.774420  189644.182530  220022.788508   
компаньон       F       146738.276481  166445.737778  187166.132477   
                M       166255.668715  213012.482232  237132.484127   
пенсионер       F       140734.920307   94674.467692  150347.841092   
                M       102621.701671   89003.094829  123759.161226   
предприниматель F                 NaN  499163.144947            NaN   
сотрудник       F       124273.280650  138432.467234  149453.841240   
                M       166542.469809  169117.452198  187776.039813   
студент         M                 NaN   98201.625314            NaN   

age_interval                    41-50          51-60          61-70  \
income_type     gender                                                
безработный     F       202722.511368            NaN            NaN   
                M                 NaN            NaN            NaN   
в декрете       F                 NaN            NaN            NaN   
госслужащий     F       159460.731526  156115.631627  189980.175554   
                M       230814.139756  212379.140158  196669.546197   
компаньон       F       189727.761047  193411.137249  190480.393270   
                M       255468.926634  219067.259298  226910.521832   
пенсионер       F       153593.775094  133838.339955  133323.160660   
                M       186971.648053  158277.757145  139608.702596   
предприниматель F                 NaN            NaN            NaN   
сотрудник       F       149706.295994  156172.829623  174436.608414   
                M       185872.825427  177101.497540  172947.136991   
студент         M                 NaN            NaN            NaN   

age_interval                    71-80  
income_type     gender                 
безработный     F                 NaN  
                M                 NaN  
в декрете       F                 NaN  
госслужащий     F       122066.732520  
                M                 NaN  
компаньон       F       191888.926159  
                M       163943.033856  
пенсионер       F       113755.472504  
                M       130485.783949  
предприниматель F                 NaN  
сотрудник       F       117130.428362  
                M       231613.310659  
студент         M                 NaN

Выборочно убедимся, что и группировка `incomes_groupby` и сводная таблица `incomes_pivot` действительно содержат одни и те же значения:

In [86]:
if incomes_pivot.loc['студент']['21-30']['M'] == incomes_groupby['студент']['M']['21-30']:
    print('Есть совпадение')
if incomes_pivot.loc['сотрудник']['31-40']['F'] == incomes_groupby['сотрудник']['F']['31-40']:
    print('Есть совпадение')
if incomes_pivot.loc['безработный']['41-50']['F'] == incomes_groupby['безработный']['F']['41-50']:
    print('Есть совпадение')

Есть совпадение
Есть совпадение
Есть совпадение


Анализируя результаты группировки в сводной таблице, можно заметить такую аномалию как очень молодой возраст клиентов, для которых типом занятости является `пенсионер`:

In [87]:
incomes_pivot.loc['пенсионер']

age_interval,0-20,21-30,31-40,41-50,51-60,61-70,71-80
gender,,,,,,,
F,140734.920307,94674.467692,150347.841092,153593.775094,133838.339955,133323.160660,113755.472504
M,102621.701671,89003.094829,123759.161226,186971.648053,158277.757145,139608.702596,130485.783949


Как видно, в наборе данных встречаются пенсионеры и в группе с возрастом 31-40, и даже в группе с возрастом 21-30. Полагаю, что пенсионеры (например, военные) могут быть начиная с возрастного диапазона 41-50 и старше, поэтому оценим количество таких "странных пенсионеров" с аномальным возрастным диапазоном: 

In [88]:
#def get_anomal_pensioners_count(df):
#    """Возращает количество 'странных пенсионеров', возраст которых попадает в один из диапазонов 21-30 или 31-40
#    """
#    anomal_pensioners = df[(df['income_type'] == 'пенсионер') & (df['age_interval'] == '21-30')]['income_type'].count()
#    anomal_pensioners += df[(df['income_type'] == 'пенсионер') & (df['age_interval'] == '31-40')]['income_type'].count()
#    return anomal_pensioners
#
#print(f'Пенсионеров с аномальным возрастным диапазоном: {get_anomal_pensioners_count(credit_dataset)}')

~~Количество "странных пенсионеров" невелико, поэтому удалим такие строки из нашей таблицы, а затем проверим, что удаление было выполнено корректно:~~

In [89]:
#credit_dataset.drop(credit_dataset[(credit_dataset['income_type'] == 'пенсионер') &
#                                   (credit_dataset['age_interval'] == '21-30')].index, inplace=True)
#credit_dataset.drop(credit_dataset[(credit_dataset['income_type'] == 'пенсионер') &
#                                   (credit_dataset['age_interval'] == '31-40')].index, inplace=True)
#
#print(f'Пенсионеров с аномальным возрастным диапазоном: {get_anomal_pensioners_count(credit_dataset)}')

~~Проверим, что удаление строк произошло корректно, для чего пересоздадим объекты и группировки и сводной таблицы (эксперимент показал, что удаление строк никак не отражается на созданной раннее сводной таблице):~~

In [90]:
#incomes_groupby, incomes_pivot = create_groupby_and_pivot(credit_dataset)
#incomes_pivot

Нам также может пригодиться инфорация о среднем (медианном) доходе по всему типу занятости. Для этих целй создадим словарь, в котором в виде ключей будут выступать строки с типом занятости, а в виде ключей - медианные доходы по всем клиентам нашего датасета, удовлетворяющим заданному типу занятости:

In [91]:
income_types = credit_dataset['income_type'].unique()

median_income_for_income_type = {}
for income_type in income_types:
    # Первый вызов median() получает медиану по полу ('gender'), второй - по возрастным интервалам ('age_interval')
    income_type_median = incomes_pivot.loc[income_type].median().median()
    median_income_for_income_type[income_type] = income_type_median

print(median_income_for_income_type)

{'сотрудник': 167789.56071048585, 'пенсионер': 136465.9316280107, 'компаньон': 206239.19827360794, 'госслужащий': 184247.38589257473, 'безработный': 131339.7516762103, 'предприниматель': 499163.1449470857, 'студент': 98201.62531401133, 'в декрете': 53829.13072905995}


Теперь создадим функцию, которая будет возвращать медианный доход для строки с описанием клиента, которая передаётся аргументом:

In [92]:
def total_income(row):
    """Возвращает медианный доход для клиента, описываемого строкой-аргументом.
    Для определения дохода, который нужно вернуть, использует информацию о типе занятости ('income_type'), поле
    ('gender') и возрастном интервале клиента ('age_interval') - данные значения применяются в виде индексов
    для группировки 'incomes_groupby', которая была создана раннее.
    
    В случае возникновения ошибки, связанной с отсутствием искомой группы в группировке, функция возвращает
    медианной значение, подсчитанное для всего типа занятости (без учёта пола и возрастного интервала).
    """
    income_type = row['income_type']
    gender = row['gender']
    age_interval = row['age_interval']
    try:
        average_total_income = incomes_groupby[income_type][gender][age_interval]
    except KeyError:
        average_total_income = median_income_for_income_type[income_type]

    return average_total_income

# Протестируем нашу функцию на нескольких примерах клиентов
print(total_income(pd.Series(data=['госслужащий', 'M', '31-40'], index=['income_type', 'gender', 'age_interval'])))
print(total_income(pd.Series(data=['сотрудник', 'F', '41-50'], index=['income_type', 'gender', 'age_interval'])))

220022.7885079367
149706.29599382734


Применим к нашей таблице метод `DataFrame.apply()`, передавая в качестве параметра функцию `total_income()`, результат будем записывать в новый столбец таблицы `averaged_total_income`:

In [93]:
credit_dataset['averaged_total_income'] = credit_dataset.apply(total_income, axis=1)

Для проверки результатов возьмём на заметку клиента-пенсионера в 12 строке, который имеет пропуск в столбце `total_income`.
Видно, что вызов `apply()` подготовил для него значение, которым можно выполнить замену пропуска; это значение
хранится в столбце `averaged_total_income`:

In [94]:
credit_dataset.iloc[11:14]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_interval,averaged_total_income
11,0,-792.701887,40,среднее,1,женат / замужем,0,F,сотрудник,0,77069.234271,покупка коммерческой недвижимости,31-40,149453.841240
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу,61-70,139608.702596
13,0,-1846.641941,54,неоконченное высшее,2,женат / замужем,0,F,сотрудник,0,130458.228857,приобретение автомобиля,51-60,156172.829623


Наконец, используя метод `Series.fillna()`, выполним замену пропусков в столбце `total_income`, пользуясь содержимым столбца `averaged_total_income`, подготовленным раннее. Убедимся, что замена пропуска действительно произошла, глядя на клиента-пенсионера из 12 строки:

In [95]:
credit_dataset['total_income'] = credit_dataset['total_income'].fillna(credit_dataset['averaged_total_income'])
credit_dataset.iloc[11:14]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_interval,averaged_total_income
11,0,-792.701887,40,среднее,1,женат / замужем,0,F,сотрудник,0,77069.234271,покупка коммерческой недвижимости,31-40,149453.841240
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,139608.702596,сыграть свадьбу,61-70,139608.702596
13,0,-1846.641941,54,неоконченное высшее,2,женат / замужем,0,F,сотрудник,0,130458.228857,приобретение автомобиля,51-60,156172.829623


**Вывод** Говоря о возможных **причинах появления пропусков** в столбцах `days_employed` и `total_income`, сложно сказать что-то определённое: пропуски наблюдаются как у клиентов со средним, так и с высшим образованием; как у клиентов возрастом до 30, так и после 60; как у женатых, так и у холостых; как у клиентов с детьми, так и у клиентов без детей. Можно было бы предположить, что пропуск в столбце `days_employed` говорит о том, что данный клиент никогда не работал, но разнообразие имеющихся при этом значений в столбце `total_income` (сотрудник, госслужащий) опровергает эту идею.

Наверное, можно сказать, что пропуск значения в одном из столбцов, `days_employed` или `total_income`, зависит от другого - в том смысле, что пропуск значения в каком-то любом из этих двух столбцов однозначно говорит о наличии пропуска и во втором тоже.

Если же рассматривать пропуски значений в `days_employed` и в `total_income` для некоторой строки как такой единый пропуск, то, на мой взгляд, такой пропуск имеет полностью случайный характер, который не зависит ни от каких других значений, и который возник, возможно, из-за какой-то проблемы технического характера в процессе подготовки датасета (могу только предположить, но может быть такое возможно, что значение хранилось в виде строки где-то в базе данных, а при экспорте данных в CSV файл возникла ошибка конвертации строки в число `float64`, в результате чего и появился пропуск).

### Замена типа данных

Как было упомянуто в [Шаге 1](#step1), содержимое столбца "стаж в днях" (`days_employed`) вряд ли представляет собой что-то осмысленное, поэтому выполним замену содержимого этого столбца, а также сменим тип данных в этом столбце - с вещественного на целочисленный.
Прежде всего создадим функцию, которая будет осуществлять замену содержимого столбца `days_employed` в зависимости от возраста (`dob_years`) клиента:

In [96]:
def calculate_days_employed(age_in_years):
    """Возвращает стаж клиента в днях, принимая возраст клиента в годах в виде параметра
    
    Стаж в днях подсчитывается в соответствии со следующей стратегией:
    - считаем, что в среднем люди начинают работать с 21 года ('employment_start_year')
    - считаем, что в среднем люди выходят на пенсию в 63 года ('employment_end_year')
    """
    employment_start_year = 21
    employment_end_year = 63

    if age_in_years < employment_start_year:
        return 0.0
    return 365.0 * (min(employment_end_year, age_in_years) - employment_start_year)

# Будем делить возвращаемое значение на 365, чтобы для простоты проверки получить значение в годах
print(calculate_days_employed(99) // 365)
print(calculate_days_employed(70) // 365)
print(calculate_days_employed(63) // 365)
print(calculate_days_employed(50) // 365)
print(calculate_days_employed(23) // 365)
print(calculate_days_employed(22) // 365)
print(calculate_days_employed(21) // 365)
print(calculate_days_employed(11) // 365)

# Также проверим возвращаемое значение в днях стажа
print(calculate_days_employed(23))

42.0
42.0
42.0
29.0
2.0
1.0
0.0
0.0
730.0


Теперь произведём заполнение столбца `days_employed`: как и прежде, воспользуемся методом `Series.apply()` и передадим ей нашу функцию `calculate_days_employed()` в качестве аргумента:

In [97]:
credit_dataset['days_employed'] = credit_dataset['dob_years'].apply(calculate_days_employed)

Проверим результаты в столбце `days_employed` на небольшом фрагменте таблицы:

In [98]:
credit_dataset.iloc[11:14]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_interval,averaged_total_income
11,0,6935.0,40,среднее,1,женат / замужем,0,F,сотрудник,0,77069.234271,покупка коммерческой недвижимости,31-40,149453.841240
12,0,15330.0,65,среднее,1,гражданский брак,1,M,пенсионер,0,139608.702596,сыграть свадьбу,61-70,139608.702596
13,0,12045.0,54,неоконченное высшее,2,женат / замужем,0,F,сотрудник,0,130458.228857,приобретение автомобиля,51-60,156172.829623


Наконец, заменим вещественный тип данных в столбце `days_employed` на целочисленный, для чего воспользуемся функцией `pandas.astype()`:

In [99]:
credit_dataset['days_employed'] = credit_dataset['days_employed'].astype(int)

Убедимся, что данные в столбце `days_employed` стали целочисленными:

In [100]:
credit_dataset.iloc[11:14]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_interval,averaged_total_income
11,0,6935,40,среднее,1,женат / замужем,0,F,сотрудник,0,77069.234271,покупка коммерческой недвижимости,31-40,149453.841240
12,0,15330,65,среднее,1,гражданский брак,1,M,пенсионер,0,139608.702596,сыграть свадьбу,61-70,139608.702596
13,0,12045,54,неоконченное высшее,2,женат / замужем,0,F,сотрудник,0,130458.228857,приобретение автомобиля,51-60,156172.829623


**Вывод** В столбце `days_employed` была выполнена замена всех существовавших ранее значений, при этом замена осуществлялась в соответствии с некоторой стратегией подсчёта трудового стажа на основании возраста клиента. Стратегия была реализована в фукнции `calculate_days_employed()`, а использование данной функции было реализовано с помощью вызова метода `Series.apply()`.

Наконец, вещественный тип данных в столбце `days_employed` был заменён на целочисленный с помощью функции `pandas.astype()`. Она была использована вместо `pandas.to_numeric()`, поскольку последняя функция переводит содержимое столбца в вещественный тип `float`, нам же требовалось получить на выходе целочисленный тип данных `int`.

### Обработка дубликатов

Как было отмечено в выводе по [Шагу 1](#step1), в таблице в столбцах `family_status` и `education` имеют место различия в регистре символов. Перед обработкой дубликатов может быть разумным сначала выполнить приведение к нижнему регистру содержимого этих двух столбцов - не исключено, что это породит новые дубликаты, которые не могли бы быть иначе обнаружены из-за разницы в регистре символов:

In [101]:
credit_dataset['family_status'] = credit_dataset['family_status'].str.lower()
credit_dataset['education'] = credit_dataset['education'].str.lower()

Подсчитаем количество дубликатов в нашей таблице с помощью `DataFrame.duplicated()` и последующим суммированием:

In [102]:
credit_dataset.duplicated().sum()

71

Дубликатов немного - всего `71`; выполним удаление дублированных строк с помощью метода `DataFrame.drop_duplicates()`, передавая аргументом значение `True` для параметра `inplace`, чтобы удаление дубликатов выполнилось "на месте":

In [103]:
credit_dataset.drop_duplicates(inplace=True)

Убедимся, что дубликатов после этого не осталось:

In [104]:
credit_dataset.duplicated().sum()

0

**Вывод** Перед удалением дубликатов было выполнено приведение к нижнему регистру содержимого столбцов  `family_status` и `education` в нашей таблице. Проверка показала, что это приведение к нижнему регистру действительно имело смысл: так, без приведения к нижнему регистру было бы обнаружено только `54` дубликата, а с ним был обнаружен `71` дубликат.

Найденные дубликаты были устранены с помощью метода `DataFrame.drop_duplicates()`.

Говоря о возможных __причинах появления дубликатов__, мне кажется, что они возникли в первую очередь из-за строк с пропусками: пропуски в категориях `days_employed` и `total_income` привели к тому, что многие клиенты без стажа и без дохода стали неразличимы между собой (остальные категории не обладают таким существенным разнообразием значений, которое могло бы помешать возникновению дубликатов).

### Лемматизация

Подключим стеммер компании "Яндекс" Mystem, а также создадим его объект:

In [105]:
from pymystem3 import Mystem
m = Mystem()

Напишем вспомогательную функцию, которая будем выполнять лемматизацию кредитных целей клиентов:

In [106]:
def lemmatize_client_purpose(purpose):
    """Лемматизирует кредитную цель клиента, описание которой передаётся аргументом-строкой.
    
    Возвращает список из лемматизированных токенов, очищенных от пробельных (whitespace)
    """
    tokens = m.lemmatize(purpose)
    return [t for t in tokens if len(t.strip()) > 0]

print(lemmatize_client_purpose('купил квартиру'))
print(lemmatize_client_purpose('на покупку машин'))
print(lemmatize_client_purpose('автомобилей'))
print(lemmatize_client_purpose('машинами'))

['купить', 'квартира']
['на', 'покупка', 'машина']
['автомобиль']
['машина']


Как и ранее, используем `Series.apply()` для вызова нашей функции `lemmatize_client_purpose()`; результат запишем в новый столбец `purpose_lemmas`:

In [107]:
credit_dataset['purpose_lemmas'] = credit_dataset['purpose'].apply(lemmatize_client_purpose)

Выборочно взглянем на результаты в столбце `purpose_lemmas`:

In [108]:
credit_dataset.iloc[11:14]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_interval,averaged_total_income,purpose_lemmas
11,0,6935,40,среднее,1,женат / замужем,0,F,сотрудник,0,77069.234271,покупка коммерческой недвижимости,31-40,149453.841240,"[покупка, коммерческий, недвижимость]"
12,0,15330,65,среднее,1,гражданский брак,1,M,пенсионер,0,139608.702596,сыграть свадьбу,61-70,139608.702596,"[сыграть, свадьба]"
13,0,12045,54,неоконченное высшее,2,женат / замужем,0,F,сотрудник,0,130458.228857,приобретение автомобиля,51-60,156172.829623,"[приобретение, автомобиль]"


**Вывод** Содержимое столбца `purpose` было лемматизировано с помощью стеммера Mystem: каждая строка с целью получения кредита была лемматизирована в список токенов, из которого были удалены пробельные токены и токены концов строк. Полученный список токенов был добавлен ещё одним столбцом `purpose_lemmas` в нашу таблицу.

### Категоризация данных

Учитывая вопросы, на которые предстоит дать ответ в контексте влияния на возврат кредита в срок, нас будут интересовать следующие категории:

- "наличие детей" (`children`)
- "семейное положение" (`family_status` или `family_status_id`)
- "уровень дохода" (`total_income`)
- "цель кредита" (`purpose` или `purpose_lemmas`)

При этом первые две категории скорее всего не нуждаются в категоризации из-за ограниченного количества значений, которое они могут принимать:

In [109]:
print(f"Уникальные значения для 'children': {credit_dataset['children'].unique()}")
print(f"Уникальные значения для 'family_status': {credit_dataset['family_status'].unique()}")
print(f"Уникальные значения для 'family_status_id': {credit_dataset['family_status_id'].unique()}")

Уникальные значения для 'children': [1 0 3 2 4 5]
Уникальные значения для 'family_status': ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'не женат / не замужем']
Уникальные значения для 'family_status_id': [0 1 2 3 4]


#### Категоризация по "уровню дохода" (`total_income`)

Категория "уровень дохода" (`total_income`), судя по всему, потребует разделения на интервалы, аналогично тому, как это было сделано с возрастом.

Определим ширину диапазона всех значений столбца `total_income` как разницу между максимальным и минимальным значением в столбце `total_income`:

In [110]:
total_income_range = credit_dataset['total_income'].max() - credit_dataset['total_income'].min()
print(f'Ширина диапазона значений "total_income": {total_income_range}')

Ширина диапазона значений "total_income": 2244936.7649294725


Чтобы разделить весь диапазон доходов, к примеру, на 7 интервалов, можно было бы попробовать сделать длину одного интервала равной примерно `300000`, однако такой подход должен хорошо показать себя при равномерном распределении доходов клиентов по всему диапазону. Однако оказалось, что доходы распределены совсем не равномерно, с подавляющим преобладанием клиентов с более низким доходом. По этой причине в качестве "ширины" интервала возьмём какое-то небольшое значение, а всех клиентов, которые получают больше `600000`, поместим в один интервал "самых богатых клиентов".

Для большей гибкости управления шириной интервала создадим отдельную функцию, которая по входному значению желаемой ширины создаст вспомогательный словарь (он будет описан ниже), необходимый для присвоения целевого "доходного интервала" клиенту по его доходу:

In [111]:
# Константа для интервала с самыми богатыми клиентами
upper_income_value = 600000

def create_quotient_to_income_interval_mapping(rounded_interval_length):
    """Строит вспомогательный словарь, который по входному частному целочисленного деления
    дохода клиента (делимое) на ширину интервала (rounded_interval_length, делитель) возвращает строку
    с именем целевого "доходного интервала" для данного клиента.
    
    К примеру, для клиента с доходом 112080 целочисленное деление на rounded_interval_length=50000 вернёт
    значение 2, и обратившись с этим значением 2 в данный словарь, мы получим имя "доходного интервала"
    для клиента с заданным доходом, в данном случае это будет '100001-150000'
    """
    quotient_to_income_interval = {}
    lower_bound = 0
    interval_index = 'a'
    while lower_bound < upper_income_value:
        quotient = lower_bound // rounded_interval_length
        upper_bound = lower_bound + rounded_interval_length
        quotient_to_income_interval[quotient] = f'({interval_index}) {lower_bound}-{upper_bound - 1}'
        interval_index = chr(ord(interval_index) + 1)
        lower_bound = upper_bound

    upper_income_interval = f'({interval_index}) {lower_bound}+'

    import json
    print(json.dumps(quotient_to_income_interval, indent=4))
    print()
    print('Самый верхний интервал (с самыми богатыми клиентами):', upper_income_interval)
    return quotient_to_income_interval, upper_income_interval

# Вызовем нашу функцию, сделав ширину интервала равной 80000, чтобы получить на выходе 8 интервалов
rounded_interval_length = 80000
quotient_to_income_interval, upper_income_interval =\
    create_quotient_to_income_interval_mapping(rounded_interval_length)

{
    "0": "(a) 0-79999",
    "1": "(b) 80000-159999",
    "2": "(c) 160000-239999",
    "3": "(d) 240000-319999",
    "4": "(e) 320000-399999",
    "5": "(f) 400000-479999",
    "6": "(g) 480000-559999",
    "7": "(h) 560000-639999"
}

Самый верхний интервал (с самыми богатыми клиентами): (i) 640000+


Теперь напишем функцию, возвращающую "доходный интервал" по переданному значению дохода клиента:

In [112]:
def get_income_interval(income):
    """Возвращает имя "доходного интервала" по доходу клиента, переданному в качестве аргумента 'income'
    Для всех доходов больше или равных 'upper_income_value' будет возвращен
    единый интервал 'upper_income_interval'.
    """
    integer_division_quotient = income // rounded_interval_length
    return quotient_to_income_interval.get(integer_division_quotient, upper_income_interval)

# Протестируем функцию на некоторых входах
print(f'Для {0} интервал "{get_income_interval(0)}"')
print(f'Для {1} интервал "{get_income_interval(1)}"')
print(f'Для {499998} интервал "{get_income_interval(499998)}"')
print(f'Для {499999} интервал "{get_income_interval(499999)}"')
print(f'Для {500000} интервал "{get_income_interval(500000)}"')
print(f'Для {599999} интервал "{get_income_interval(599999)}"')
print(f'Для {600000} интервал "{get_income_interval(600000)}"')
print(f'Для {600001} интервал "{get_income_interval(600001)}"')

Для 0 интервал "(a) 0-79999"
Для 1 интервал "(a) 0-79999"
Для 499998 интервал "(g) 480000-559999"
Для 499999 интервал "(g) 480000-559999"
Для 500000 интервал "(g) 480000-559999"
Для 599999 интервал "(h) 560000-639999"
Для 600000 интервал "(h) 560000-639999"
Для 600001 интервал "(h) 560000-639999"


Применим нашу функцию через `Series.apply()`, создавая новый столбец `income_interval`. Сделаем группировку по столбцу `income_interval`, результаты сохраним в переменных для последующих ответов на вопросы в следующем разделе:

In [113]:
credit_dataset['income_interval'] = credit_dataset['total_income'].apply(get_income_interval)
income_intervals_counts_80k = credit_dataset.groupby('income_interval')['income_interval'].count()
income_intervals_means_80k = credit_dataset.groupby('income_interval')['debt'].mean() * 100

Теперь построим более узкие интервалы, шириной в `25000`, перезапишем значение в столбце `income_interval`. Опять применим функцию к таблице и сохраним результаты в переменных:

In [114]:
rounded_interval_length = 25000
quotient_to_income_interval, upper_income_interval =\
    create_quotient_to_income_interval_mapping(rounded_interval_length)

credit_dataset['income_interval'] = credit_dataset['total_income'].apply(get_income_interval)
income_intervals_counts_25k = credit_dataset.groupby('income_interval')['income_interval'].count()
income_intervals_means_25k = credit_dataset.groupby('income_interval')['debt'].mean() * 100

{
    "0": "(a) 0-24999",
    "1": "(b) 25000-49999",
    "2": "(c) 50000-74999",
    "3": "(d) 75000-99999",
    "4": "(e) 100000-124999",
    "5": "(f) 125000-149999",
    "6": "(g) 150000-174999",
    "7": "(h) 175000-199999",
    "8": "(i) 200000-224999",
    "9": "(j) 225000-249999",
    "10": "(k) 250000-274999",
    "11": "(l) 275000-299999",
    "12": "(m) 300000-324999",
    "13": "(n) 325000-349999",
    "14": "(o) 350000-374999",
    "15": "(p) 375000-399999",
    "16": "(q) 400000-424999",
    "17": "(r) 425000-449999",
    "18": "(s) 450000-474999",
    "19": "(t) 475000-499999",
    "20": "(u) 500000-524999",
    "21": "(v) 525000-549999",
    "22": "(w) 550000-574999",
    "23": "(x) 575000-599999"
}

Самый верхний интервал (с самыми богатыми клиентами): (y) 600000+


In [115]:
rounded_interval_length = 150000
quotient_to_income_interval, upper_income_interval =\
    create_quotient_to_income_interval_mapping(rounded_interval_length)

credit_dataset['income_interval'] = credit_dataset['total_income'].apply(get_income_interval)
income_intervals_counts_150k = credit_dataset.groupby('income_interval')['income_interval'].count()
income_intervals_means_150k = credit_dataset.groupby('income_interval')['debt'].mean() * 100

{
    "0": "(a) 0-149999",
    "1": "(b) 150000-299999",
    "2": "(c) 300000-449999",
    "3": "(d) 450000-599999"
}

Самый верхний интервал (с самыми богатыми клиентами): (e) 600000+


#### Категоризация по "цели кредита" (`purpose`/`purpose_lemmas`)

Категория "цель кредита" (`purpose` или `purpose_lemmas`) потребует разделения на категории в зависимости от цели: "автомобиль", "жильё", "свадьба" и прочее. Посмотрим на все уникальные значения целей кредита:

In [116]:
credit_dataset['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Получившийся список невелик, из него можно выделить следущие категории, отражающие цель кредита:

- жильё/недвижимость
- автомобиль
- образование
- свадьба

Напишем функцию, которая поможет выполнить категоризацию:

In [117]:
def detect_purpose_category(purpose_lemmas):
    """Возвращает категорию для переданной аргументом цели кредита, представленной списком лемматизированных
    токенов
    
    Определение категории выполняется на основе нахождения среди лемм определённых слов-маркеров
    """
    
    if 'жильё' in purpose_lemmas or 'жилье' in purpose_lemmas or 'недвижимость' in purpose_lemmas:
        return 'недвижимость'
    if 'автомобиль' in purpose_lemmas:
        return 'автомобиль'
    if 'образование' in purpose_lemmas:
        return 'образование'
    if 'свадьба' in purpose_lemmas:
        return 'свадьба'

    print('Категория не присвоена: недостаточно слова-маркеров')
    return None

print(detect_purpose_category(['на', 'покупка', 'жилье']))
print(detect_purpose_category(['жильё']))
print(detect_purpose_category(['сделка', 'с', 'автомобиль']))
print(detect_purpose_category(['дополнительный', 'образование']))
print(detect_purpose_category(['сыграть', 'свадьба']))
print(detect_purpose_category(['на', 'путешествие']))

недвижимость
недвижимость
автомобиль
образование
свадьба
Категория не присвоена: недостаточно слова-маркеров
None


Как и ранее, применим функцию с помощью `Series.apply()` и проверим, что все кредитные цели клиентов получили назначенную функцией категорию. Наконец, сделаем группировку в зависимости от категории с целью кредита, сохраняя результаты для следующего раздела:

In [118]:
credit_dataset['purpose_category'] = credit_dataset['purpose_lemmas'].apply(detect_purpose_category)

print(f"Число пропусков в столбце 'purpose_category' : {credit_dataset[credit_dataset['purpose_category'].isnull()].size}")

purpose_category_counts = credit_dataset.groupby('purpose_category')['income_interval'].count()
purpose_category_means = credit_dataset.groupby('purpose_category')['debt'].mean() * 100

Число пропусков в столбце 'purpose_category' : 0


**Вывод**

В следующем разделе нужно будет исследовать следующие 4 столбца на их влияние на возврат кредита в срок:

- "наличие детей" (`children`)
- "семейное положение" (`family_status`/`family_status_id`)
- "уровень дохода" (`total_income`)
- "цель кредита" (`purpose`/`purpose_lemmas`)

Из них столбцы "уровень дохода" и "цель кредита", на мой взгляд, нуждались в категоризации, которая и была выполнена в данном разделе.

Столбец `цель кредита` был категоризован на крупные категории (`недвижимость`, `автомобиль`, `образование`, `свадьба`).

Что касается `уровня дохода`, категоризация этого столбца была выполнена разделением полного диапазона доходов на интервалы-категории. Данное разделение на интервалы было выполнено в двух вариантах:

- с шириной "доходного интервала" в `80000` (число интервалов `9`, с учётом самого верхнего интервала)
- с шириной "доходного интервала" в `25000` (число интервалов `25`, с учётом самого верхнего интервала)

## Шаг 3. Ответьте на вопросы

Сначала исследуем какой процент клиентов нашего набора данных имеет просроченный кредит:

In [119]:
clients_without_debt_count = credit_dataset[credit_dataset['debt'] == 0]['debt'].count()
print(f"Число клиентов без долга по кредиту   : {clients_without_debt_count}")

clients_with_debt_count = credit_dataset[credit_dataset['debt'] == 1]['debt'].count()
print(f"Число клиентов с долгом по кредиту    : {clients_with_debt_count}")

clients_with_debt_percentage = (clients_with_debt_count / (clients_with_debt_count + clients_without_debt_count))
print(f"Процент клиентов с долгом по кредиту  : {clients_with_debt_percentage:.2%}")

Число клиентов без долга по кредиту   : 19713
Число клиентов с долгом по кредиту    : 1741
Процент клиентов с долгом по кредиту  : 8.12%


Прежде чем отвечать на следующие вопросы, стоит подчеркнуть способ определения зависимости между возвратом кредита в срок и той или иной категорией.

Определение зависимости будет происходить после выполнения группировки с последующим применением агрегирующей функции `mean()`. Домножив результат на 100, мы получим набор значений в процентах, каждое из которых (`X`) можно интерпретировать следующим образом:

"_X представляет собой процент клиентов __с долгом__ в рамках текущей категории_"

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [120]:
print('Количество клиентов, попавших в каждую из категорий по количеству детей:')
print(credit_dataset.groupby('children')['debt'].count())
print()
print('Средние значения столбца "debt" по категориям количества детей:')
print(credit_dataset.groupby('children')['debt'].mean() * 100)

Количество клиентов, попавших в каждую из категорий по количеству детей:
children
0    14214
1     4808
2     2052
3      330
4       41
5        9
Name: debt, dtype: int64

Средние значения столбца "debt" по категориям количества детей:
children
0    7.541860
1    9.234609
2    9.454191
3    8.181818
4    9.756098
5    0.000000
Name: debt, dtype: float64


In [121]:
# КОД РЕВЬЮЕРА

# str - чтобы добавить %
credit_dataset.groupby('children')['debt'].agg(['count', 'sum', lambda x: str(round(x.mean()*100,2)) +'%' ])

,count,sum,<lambda_0>
children,,,
0,14214,1072,7.54%
1,4808,444,9.23%
2,2052,194,9.45%
3,330,27,8.18%
4,41,4,9.76%
5,9,0,0.0%


In [122]:
# КОД РЕВЬЮЕРА

# Или просто форматируем вывод
credit_dataset.groupby('children')['debt'].agg(['count', 'sum', lambda x: '{:.2%} '.format(x.mean())])

,count,sum,<lambda_0>
children,,,
0,14214,1072,7.54%
1,4808,444,9.23%
2,2052,194,9.45%
3,330,27,8.18%
4,41,4,9.76%
5,9,0,0.00%


In [123]:
# КОД РЕВЬЮЕРА

credit_dataset.groupby('children').agg({'debt':['count', 'sum', 'mean'], 'total_income': 'median'})

debt                   total_income
          count   sum      mean         median
children                                      
0         14214  1072  0.075419  148138.531030
1          4808   444  0.092346  149706.295994
2          2052   194  0.094542  149453.841240
3           330    27  0.081818  160665.624794
4            41     4  0.097561  158070.269384
5             9     0  0.000000  185872.825427

In [124]:
# КОД РЕВЬЮЕРА

qq = credit_dataset.groupby('children').agg({'debt':['count', 'sum', 'mean'], 'total_income': 'median'})

qq.columns = qq.columns.droplevel(0)
qq

,count,sum,mean,median
children,,,,
0,14214,1072,0.075419,148138.531030
1,4808,444,0.092346,149706.295994
2,2052,194,0.094542,149453.841240
3,330,27,0.081818,160665.624794
4,41,4,0.097561,158070.269384
5,9,0,0.000000,185872.825427


**Вывод**

На мой взгляд, зависимость между наличием детей и возвратом кредита в срок довольно слаба. Видно, что меньше всего клиентов с просроченным кредитом среди тех клиентов, которые не имеют детей, `7.53%` (делать вывод о 100% надёжности клиентов с `5` детьми кажется неразумным, поскольку таких клиентов оказалось слишком мало, всего `9`). Заметим, однако, что среди клиентов с `4` детьми отмечается наиболее высокий процент должников - `9.75%` (и это при том, что их количество не сильно отличается от количества клиентов с `5` детьми). Сделать вывод о росте процента должников с увеличением количества детей также нельзя, поскольку рост с `0` до `2` детей прекращается на количестве детей равном `3`, где наблюдается заметный спад, до `8.2%`.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [125]:
print('Количество клиентов, попавших в каждую из категорий по семейному положению:')
print(credit_dataset.groupby('family_status')['debt'].count())
print()
print('Средние значения столбца "debt" по категориям семейного положения:')
print(credit_dataset.groupby('family_status')['debt'].mean().sort_values() * 100)

Количество клиентов, попавших в каждую из категорий по семейному положению:
family_status
в разводе                 1195
вдовец / вдова             959
гражданский брак          4151
женат / замужем          12339
не женат / не замужем     2810
Name: debt, dtype: int64

Средние значения столбца "debt" по категориям семейного положения:
family_status
вдовец / вдова           6.569343
в разводе                7.112971
женат / замужем          7.545182
гражданский брак         9.347145
не женат / не замужем    9.750890
Name: debt, dtype: float64


In [126]:
# КОД РЕВЬЮЕРА

dict(zip(credit_dataset['family_status_id'], credit_dataset['family_status']))

{0: 'женат / замужем',
 1: 'гражданский брак',
 2: 'вдовец / вдова',
 3: 'в разводе',
 4: 'не женат / не замужем'}

In [127]:
# КОД РЕВЬЮЕРА

print('Создали словарь:')
family_dict = credit_dataset[['family_status_id', 'family_status']]
family_dict = family_dict.drop_duplicates().reset_index(drop=True)
display(family_dict)


print('\n\nСгруппированная таблица. Берем по id, другой столбец удалили:')
a = credit_dataset.groupby('family_status_id')['debt'].agg(['count', 'sum', lambda x: '{:.2%} '.format(x.mean())])
display(a)


# Заменяем
print('\n\nЗаменяем численные значения по ключу словаря:')
a.reset_index().replace({'family_status_id': family_dict.family_status.to_dict()})

Создали словарь:


,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем




Сгруппированная таблица. Берем по id, другой столбец удалили:


,count,sum,<lambda_0>
family_status_id,,,
0,12339,931,7.55%
1,4151,388,9.35%
2,959,63,6.57%
3,1195,85,7.11%
4,2810,274,9.75%




Заменяем численные значения по ключу словаря:


,family_status_id,count,sum,<lambda_0>
0,женат / замужем,12339,931,7.55%
1,гражданский брак,4151,388,9.35%
2,вдовец / вдова,959,63,6.57%
3,в разводе,1195,85,7.11%
4,не женат / не замужем,2810,274,9.75%


**Вывод**

Полагаю, что результаты для столбца "семейное положение" можно интерпретировать как имеющие определённую зависимость: можно сделать вывод, что процент должников по кредиту снижается по мере того как человек проживает определённые этапы (семейной) жизни.

Наибольший процент должников наблюдается среди незамужних/неженатых клиентов - `9.77%` (что говорит, возможно, об их недостаточной серьёзности). Как только человек достигает этапа гражданского брака, то он попадает в другую группу клиентов, с несколько сниженным процентов должников по кредиту - `9.35%`. После женитьбы вновь происходит снижение процента должников, причём довольно существенно, до `7.52%`. У той группы клиентов, кто развёлся после женитьбы, вновь наблюдается снижение процента должников - `7.14%`. Наконец, минимальный процент должников в целом отмечается среди вдов и вдовцов - `6.47%`.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [128]:
print('Количество клиентов, попавших в каждую из категорий по доходу (80 000):')
print(income_intervals_counts_80k)
print()
print('Средние значения столбца "debt" по категориям дохода (80 000):')
print(income_intervals_means_80k)
print()
print('=' * 100)
print()
print('Количество клиентов, попавших в каждую из категорий по доходу (25 000):')
print(income_intervals_counts_25k)
print()
print('Средние значения столбца "debt" по категориям дохода (25 000):')
print(income_intervals_means_25k)

Количество клиентов, попавших в каждую из категорий по доходу (80 000):
income_interval
(a) 0-79999          2276
(b) 80000-159999     9986
(c) 160000-239999    6024
(d) 240000-319999    1945
(e) 320000-399999     694
(f) 400000-479999     262
(g) 480000-559999     125
(h) 560000-639999      47
(i) 640000+            95
Name: income_interval, dtype: int64

Средние значения столбца "debt" по категориям дохода (80 000):
income_interval
(a) 0-79999          7.644991
(b) 80000-159999     8.491889
(c) 160000-239999    8.283533
(d) 240000-319999    6.940874
(e) 320000-399999    7.780980
(f) 400000-479999    5.725191
(g) 480000-559999    6.400000
(h) 560000-639999    6.382979
(i) 640000+          5.263158
Name: debt, dtype: float64


Количество клиентов, попавших в каждую из категорий по доходу (25 000):
income_interval
(a) 0-24999             8
(b) 25000-49999       364
(c) 50000-74999      1493
(d) 75000-99999      2599
(e) 100000-124999    2918
(f) 125000-149999    3693
(g) 150000-174999  

**Вывод**

~~Глядя на процент должников в категоризации с шириной "доходного интервала" в `80 000`, я бы воздержался от выводов о том, что уровень дохода очень ярко свидетельствует о надёжности заёмщика. Проходя по получившимся интервалам от меньшего (по доходу) к большему и рассматривая пару соседних, можно увидеть чередование трендов "подъём", "спуск", "подъём", "спуск". В то же время можно заметить, что в целом у `4` последних интервалов процент должников ниже, чем у первых `4`. Так что, возможно, осторожно можно утверждать, что:~~

~~по мере увеличения доходов процент должников по кредитам в некоторой степени снижается~~

~~Что касается процентов должников в категоризации с более узкими "доходными интрвалами" в `25 000`, то сразу же бросается в глаза больший процент должников среди клиентов с самой низкой зарплатой (`0-24999`) - `14.28%`. Однако таких клиентов всего `7` поэтому, возможно, их можно посчитать выбросами. Также хочется отметить некоторые всплески ближе к концу списка диапазонов - интервалы `(o)`, `(u)` (`186` и `43` клиента соответственно), которые имеют процент должников выше `9%`. Возможно, что данные интервалы являются результатом деления исходного диапазона доходов на слишком узкие "доходные интервалы", тогда как при более широких интервалах данные всплески были бы усреднены. Тем не менее, и для данной категоризации я бы по-прежнему придерживался того утверждения, которое было сделано в данном выводе выше.~~

In [129]:
print('Количество клиентов, попавших в каждую из категорий по доходу (150 000):')
print(income_intervals_counts_150k)
print()
print('Средние значения столбца "debt" по категориям дохода (150 000):')
print(income_intervals_means_150k)

Количество клиентов, попавших в каждую из категорий по доходу (150 000):
income_interval
(a) 0-149999         11075
(b) 150000-299999     8896
(c) 300000-449999     1150
(d) 450000-599999      223
(e) 600000+            110
Name: income_interval, dtype: int64

Средние значения столбца "debt" по категориям дохода (150 000):
income_interval
(a) 0-149999         8.316027
(b) 150000-299999    8.026079
(c) 300000-449999    7.652174
(d) 450000-599999    4.484305
(e) 600000+          7.272727
Name: debt, dtype: float64


In [130]:
credit_dataset['total_income_quantiles'] = pd.qcut(credit_dataset['total_income'], 5)
credit_dataset.groupby('total_income_quantiles')['debt'].agg(['count', 'sum', lambda x: f'{x.mean():.2%}'])

,count,sum,<lambda_0>
total_income_quantiles,,,
"(20667.263, 98532.445]",4291,344,8.02%
"(98532.445, 133838.34]",4417,365,8.26%
"(133838.34, 165856.104]",4164,364,8.74%
"(165856.104, 217381.214]",4290,363,8.46%
"(217381.214, 2265604.029]",4291,305,7.11%


**Вывод**

Глядя на таблицу с результатами группировки по квантилям, можно видеть, что наибольший процент должников наблюдается у клиентов средним доходом - `8.74%` в центральной категории. При этом значения в крайних категориях по доходу (клиенты с самым большим и с самым маленьким доходом) меньше, чем значения в трёх категориях в середине. Думаю, это можно считать подтверждением гипотезы о том, что люди со средним доходом хуже всех выплачивают кредит. Люди с наибольшим доходом выглядят наиболее надёжными, их процент должников - `7.11%`.

- Как разные цели кредита влияют на его возврат в срок?

In [132]:
print('Количество клиентов, попавших в каждую из категорий по цели кредита:')
print(purpose_category_counts)
print()
print('Средние значения столбца "debt" по категориям цели кредита:')
print(purpose_category_means.sort_values())

Количество клиентов, попавших в каждую из категорий по цели кредита:
purpose_category
автомобиль       4306
недвижимость    10811
образование      4013
свадьба          2324
Name: income_interval, dtype: int64

Средние значения столбца "debt" по категориям цели кредита:
purpose_category
недвижимость    7.233373
свадьба         8.003442
образование     9.220035
автомобиль      9.359034
Name: debt, dtype: float64


**Вывод**

Можно видеть, что наименьший процент должников по кредитам наблюдается у клиентов, взявших кредит на "__недвижимость__" (коммерческую или жилую) - `7.23%`. Полагаю, что это связано с большим кредитом и вытекающей из этого серьёзности ситуации. Можно сделать гипотезу о том, что недвижимостью интересуются уже бывшие/находящиеся в браке люди, а они, как мы видели выше, демонстрируют более высокую надёжность по сравнению с неженатыми клиентами и клиентами, живущими в гражданском браке.

Аналогичным образом, полагаю, можно интерпретировать и проценты по остальным категориям. Так, кредит на "__свадьбу__" берут люди, которые, думаю, уже обладают определённой степенью надёжности и серьёзности (раз они готовятся вступить в семейную жизнь), поэтому и процент должников по ним тоже сравнительно невысок - `7.99%`.

В то же время кредит на "__образование__" могут брать ещё довольно молодые люди, не обладающие пока высокой степенью надёжности, поэтому и процент должников среди них выше - `9.22%`.

Оставшаяся категория "__автомобиль__" обладает самым высоким процентом должников - `9.3%`. Сложно сказать наверняка, связано ли это с тем, что автомобили в кредит покупают по большей части молодые неженатые люди, или же с чем-то другим. Это было бы интересно исследовать отдельно.

## Шаг 4. Общий вывод
<a id='final_outcome'></a>

В ходе работы были исследованы зависимости между возвратом кредита в срок и следующими категориями:

- "наличие детей"
- "семейное положение"
- "уровень дохода"
- "цель кредита"

Результат анализа по выявлению самых надёжных и самых безответственных клиентов:

- Категория "__количества детей__":

    - самые надёжные: клиенты без детей, **7.53%**
    
    - самые безответственные: клиенты с 4 детьми, **9.75%**


- Категория "__семейное положение__":

    - самые надёжные: вдовы и вдовцы, **6.47%**
    
    - cамые безответственные: незамужние/неженатые клиенты, **9.77%**


- Категория "__уровень дохода__":

    - самые надёжные: клиенты с самым высоким доходом, **7.11%**
    
    - cамые безответственные: клиенты со средним доходом, **8.74%**
    

- Категория "__цель кредита__":

    - самые надёжные: клиенты, взявшие кредит на недвижимость, **7.23%**
    
    - cамые безответственные: клиенты, взявшие кредит на автомобидь, **9.3%**
    
Также можно отметить, что общее количество ненадёжных клиентов по всем описанным 4 категориям весьма слабо отличается от среднего процента должников во всём наборе данных (**8.11%**).

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.